# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import concurrent.futures

from claudette import *
from fastcore.utils import *
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm

In [ ]:
#| export
class FastData:
    def __init__(self, calls: int = 100, period: int = 60):
        self.set_rate_limit(calls, period)

    def set_rate_limit(self, calls: int, period: int):
        """Set a new rate limit."""
        @sleep_and_retry
        @limits(calls=calls, period=period)
        def rate_limited_call(model: str, prompt: str, response_model, sp: str):
            chat = Chat(
                model,
                sp=sp,
                tools=[response_model],
                tool_choice={'response_model': response_model},
            )
            chat(prompt, temp=1)
            return chat.last_tool_result.content
        
        self._rate_limited_call = rate_limited_call

    def generate(self, 
                 prompt_template: str, 
                 inputs: list[dict], 
                 response_model, 
                 model: str = "claude-3-haiku-20240307",
                 sp: str = "You are a helpful assistant.",
                 max_workers: int = 64) -> list[dict]:
        
        def process_input(input_data):
            try:
                prompt = prompt_template.format(**input_data)
                response = self._rate_limited_call(
                    model=model,
                    prompt=prompt,
                    response_model=response_model,
                    sp=sp
                )
                return response
            except Exception as e:
                print(f"Error processing input: {e}")
                return None

        results = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [executor.submit(process_input, input_data) for input_data in inputs]
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(inputs)):
                result = future.result()
                results.append(result)
        
        return results

In [ ]:
class Translation():
    """Translation from an English phrase to a German phrase"""
    def __init__(self, english: str, german: str): store_attr()
    
    __repr__ = basic_repr(["english", "german"])
    def __str__(self): return "The translation has been created."

Translation("Hello, how are you today?", "Hallo, wie geht es Ihnen heute?")

__main__.Translation(english='Hello, how are you today?', german='Hallo, wie geht es Ihnen heute?')

In [ ]:
from datasets import load_dataset

examples = [
    Translation(
        english="Hello, my name is Nathan. I am a research scientist at an AI startup.",
        german="Hallo mein Name ist Nathan. Ich bin wissenschaftlicher Mitarbeiter bei einem KI-Startup."),
    Translation(
        english="How much wood could a woodchuck chuck if a woodchuck could chuck wood?",
        german="Wie viel Holz könnte ein Waldmurmeltier einspannen, wenn ein Waldmurmeltier Holz einspannen könnte?"),
    Translation(
        english="Thomas Cranmer (2 July 1489 - 21 March 1556) was a leader of the English Reformation and Archbishop of Canterbury during the reigns of Henry VIII, Edward VI and, for a short time, Mary I. He helped build the case for the annulment of Henry's marriage to Catherine of Aragon, which was one of the causes of the separation of the English Church from union with the Holy See.",
        german="Thomas Cranmer (2. Juli 1489 - 21. März 1556) war ein Anführer der englischen Reformation und Erzbischof von Canterbury während der Herrschaft von Heinrich VIII., Eduard VI. und für kurze Zeit auch Maria I. Er half bei der Ausarbeitung der Klage für die Aufhebung von Heinrichs Heirat mit Katharina von Aragon, die eine der Ursachen für die Trennung der englischen Kirche von der Union mit dem Heiligen Stuhl war."
    ),
]
examples = "\n- ".join([f"{e.english} -> {e.german}" for e in examples])

# Load personas
personas = load_dataset("proj-persona/PersonaHub", "persona", split='train').select(range(3))['persona']

sp = "You will help generate synethetic data of English and German phrases."
prompt_template = """\
Here are some examples:
{examples}

Create an english and german translation pair that is similar to the examples and would be appropriate for the following persona: {persona}
"""

# Generate translations
fast_data = FastData()
translations = fast_data.generate(
    prompt_template=prompt_template,
    inputs=[{"persona": persona, "examples": examples} for persona in personas],
    response_model=Translation,
    model="claude-3-haiku-20240307",
    sp=sp
)

print("Translations:")
print(translations)

100%|████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.27it/s]

Translations:
[__main__.Translation(english='Proper documentation is critical for any legal contract or agreement. Incomplete or inaccurate project details can lead to significant issues down the line.', german='Ordnungsgemäße Dokumentation ist entscheidend für jeden Rechtsvertrag oder jede Vereinbarung. Unvollständige oder ungenaue Projektdetails können später zu erheblichen Problemen führen.'), __main__.Translation(english="Postpartum complications can be life-threatening, but awareness and education can save lives. As a maternal health advocate, I'm dedicated to empowering women and their families to recognize warning signs and seek medical care.", german='Komplikationen nach der Geburt können lebensbedrohlich sein, aber Bewusstsein und Bildung können Menschenleben retten. Als Fürsprecherin für die Gesundheit von Müttern setze ich mich entschlossen dafür ein, Frauen und ihre Familien zu befähigen, Warnsignale zu erkennen und ärztliche Hilfe in Anspruch zu nehmen.'), __main__.Transla

In [ ]:
class TranslationCritique():
    """
    A critique of the translation.
    """
    def __init__(
        self,
        critique: str, # A critique of the translation.
        score: int # A score of the translation from 1 to 5. 
    ): store_attr()
        
    __repr__ = basic_repr(['critique', 'score'])
    def __str__(self): return "The critique has been created."


sp = "You will help critique synethetic data of English and German phrases."
critique_template = """\
Below is an extract of a translation. Evaluate its quality as a senior translator would, considering its suitability for professional use. Use the additive 5-point scoring system described below. Points are accumulated based on the satisfaction of each criterion:

- Add 1 point if the translation conveys the basic meaning of the source text, even if it includes some minor errors or awkward phrasing.
- Add another point if the translation is generally accurate but lacks refinement in style or fails to capture some nuances of the original. It might use inconsistent terminology or have occasional lapses in register.
- Award a third point if the translation is appropriate for professional use and accurately conveys key concepts of the source text. It demonstrates good understanding of both languages, though it may not be flawless or could include some slight inconsistencies. It resembles the work of a competent translator but may have room for improvement in fluency or precision.
- Grant a fourth point if the translation is highly accurate and reads naturally in the target language, exhibiting a consistent and appropriate style. It could be similar to the work of an experienced translator, offering faithful rendering of content and tone, with minimal errors, and effectively handling complex concepts or cultural references. The result is coherent, well-expressed, and valuable for its intended purpose.
- Bestow a fifth point if the translation is outstanding, demonstrating mastery of both source and target languages. It captures subtle nuances, maintains the author's voice and intent, and reads as if it were originally written in the target language. The translator has made excellent choices in dealing with challenging elements like wordplay, idiomatic expressions, or culture-specific content.

The translation extract:
{translation}

After examining the translation:

- Briefly justify your total score, up to 100 words.
- Conclude with the score of the translation.
"""

critiques = fast_data.generate(
    prompt_template=critique_template,
    inputs=[{"translation": f"{t.english} -> {t.german}"} for t in translations],
    response_model=TranslationCritique,
    model="claude-3-5-sonnet-20240620",
    sp=sp
)

print("Critiques:")
print(critiques)

100%|████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.31s/it]

Critiques:
[__main__.TranslationCritique(critique='The translation accurately conveys the main message and maintains the tone of the original text. It demonstrates a good understanding of both languages and handles the medical terminology appropriately. The phrasing is natural in German and captures the call to action effectively. There are minor improvements possible: "Komplikationen nach der Geburt" could be more precisely "Postpartale Komplikationen," and "eine gesunde Genesung haben" could be more idiomatic as "sich gut erholen." However, these are minor points in an otherwise excellent translation that would be suitable for professional use.', score=4), __main__.TranslationCritique(critique='This translation is excellent, deserving a full 5 points. It accurately conveys the original meaning, maintaining both the content and tone of the source text. The German version captures the nuances of "marked by" with "geprägt" and "authoritarian rule" with "autokratischen Herrschaft". The p

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()